In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import datetime as dt

In [2]:
def get_obj_val(d,i_a):
    # input: a vector d which is the demand from 
    val = 0
    for i in range(len(d)):
        val = val + np.maximum(h*(i_a -np.sum(d[0:i+1])),0) + np.maximum(b*(np.sum(d[0:i+1])-i_a),0)
        #print('val updated to', val)
    return val
    
def opt_order(d, inv):
    pre_val = 10000000
    pre_order = -100
    if len(d) == 0:
#        print('d zero length!')
        return 0
    for i in (np.array(range(len(d)))[::-1]):
        #print(i)
        temp_val = get_obj_val( d , np.sum(d[0:i+1]))
        temp_order = np.sum(d[0:i+1]) -inv
        if (temp_val > pre_val ):
#            print('checked point ',i,'val is', temp_val)
#            print('temp_val', temp_val,'temp_order', temp_order)
#            print('pre_val', pre_val,'pre_order', pre_order)
            return np.maximum(pre_order,0)
        else:
#            print('checked point ',i,'val is', temp_val)
            pre_val = temp_val
            pre_order = temp_order
#    print('keep decreasing')  
    return np.maximum(temp_order,0)

In [3]:
df_sales = pd.read_csv('../data/1320/rdc_sales_1320_replenishment_V1_filled_pp.csv', index_col=0)
# df_orders = pd.read_csv('total.csv', error_bad_lines = False)

In [4]:
df_sales = df_sales.rename(columns=lambda x : (dt.datetime(2016,1,1) + dt.timedelta(days=int(x)-730)).date())

In [5]:
df_sales = df_sales.iloc[:,-62:].reset_index()

In [72]:
df_orders = pd.read_csv('../data/1320_feature/features_v11.csv')

In [73]:
df_orders_orig = df_orders
df_orders_orig['overall_opt_order'] = -1*np.ones(len(df_orders_orig))
df_orders_orig['inv'] = np.zeros(len(df_orders_orig))
df_orders_orig['d_vlt'] = np.zeros(len(df_orders_orig))
df_orders_orig['initial_stock_overall'] = np.zeros(len(df_orders_orig))

df_sales_orig = df_sales

#####get unique (sku,dc) pair
df_pair1 = df_orders[['item_sku_id']].drop_duplicates()
df_pair2 = df_sales[['row']].drop_duplicates()
df_pair2.columns = ['item_sku_id']

pair_list = pd.merge(df_pair1, df_pair2, how='inner', on=['item_sku_id']).values

In [74]:
df_orders_orig = df_orders_orig[df_orders_orig['item_sku_id'].isin(pair_list.T[0])]

In [75]:
h = 1
b = 9

# START_DAY = 547 #the first day of calculation period with reference to 2016/01/01
START_DAY = 912

In [76]:
df_orders_orig.columns

Index(['item_sku_id', 'int_org_num', 'pur_bill_id', 'item_first_cate_cd',
       'item_second_cate_cd', 'item_third_cate_cd', 'brand_code',
       'create_day_of_week', 'create_tm', 'complete_dt',
       ...
       'eq', 'next_complete_dt', 'demand_RV', 'demand_V', 'target_decision',
       'target_decision_nobc', 'overall_opt_order', 'inv', 'd_vlt',
       'initial_stock_overall'],
      dtype='object', length=142)

In [77]:
row_index = 0
for i in tqdm(range(len(pair_list))): #

    sku_dc = pair_list[i][0]
#     if(i%100==0):
#         print(i)
    
    df_orders = df_orders_orig[df_orders_orig.item_sku_id == sku_dc ]

    if(len(df_orders)!=0):
        order_time = np.array(df_orders['create_tm_index'])-START_DAY
        arrive_time = np.array(df_orders['complete_dt_index'])-START_DAY
        
        init_inv0 = np.array(df_orders['initial_stock'])
        
#         demand = df_sales_orig[df_sales_orig.sku_id == sku]
#         demand = demand[demand.int_org_num == dc].values
#         demand = np.delete(demand, 0)
#         demand = np.delete(demand, 0)        
        demand = df_sales_orig[df_sales_orig.row == sku_dc].values[0][1:]

        #demand = demand[366:]
        init_inv = []
        for i in range(len(order_time)):
            init_inv.append(init_inv0[i] - np.sum(demand[order_time[i]:arrive_time[i]]))
        init_inv = np.array(init_inv) 
    
#         hist_order = df_orders['actual_pur_qtty'].values
        #hist_target = df_orders['target_decision'].values
    
        #######compute inv level at v1    
        #print(init_inv)     
        inv = init_inv[0]
    
        order_list = []
        
#         row_index = df_orders_orig[(df_orders_orig['sku_id'] == sku )& (df_orders_orig['int_org_num'] == dc)]\
#                                 .index.values[0]#.astype(int)
        
#         print(row_index)
    
        for i in range(len(order_time)):
            #print('computing ', i, ' th order')
            if i == len(order_time)-1:
                d = demand[arrive_time[i]:]
                #print('last d', d)
            else:
                d = demand[arrive_time[i]:arrive_time[i+1]]
#               print('d is', d)
#               print('inv is', inv)
            order = opt_order(d,inv)
            # print(i, ' th order is', order)
            order_list.append(order)
            
            df_orders_orig.iloc[row_index, -4]= order
            df_orders_orig.iloc[row_index, -3]= inv
            df_orders_orig.iloc[row_index, -2]= np.sum(demand[order_time[i]:arrive_time[i]])
            df_orders_orig.iloc[row_index, -1]= inv+np.sum(demand[order_time[i]:arrive_time[i]])
            
#           #if (i%3 == 2) and (i >=2) and (inv == 0)and (i+1 < len(order_time)):
#           #print('NOTICE !!! USE HIST INV AGAIN !!!!!!!')
#           #inv = init_inv[i+1]
#        #else:
            inv = inv + order - np.sum(d)
            row_index += 1

100%|██████████| 32881/32881 [22:37<00:00, 24.22it/s]


In [79]:
# df_orders_orig = df_orders_orig.sort_values(['sku_id', 'int_org_num', 'create_tm_index'], ascending=[True, True, True])
df_orders_orig.to_csv('../data/1320_feature/features_v12.csv', index=False)

In [78]:
df_orders_orig[df_orders_orig['item_sku_id']=='319993#4']

,item_sku_id,int_org_num,pur_bill_id,item_first_cate_cd,item_second_cate_cd,item_third_cate_cd,brand_code,create_day_of_week,create_tm,complete_dt,...,eq,next_complete_dt,demand_RV,demand_V,target_decision,target_decision_nobc,overall_opt_order,inv,d_vlt,initial_stock_overall
465,319993#4,4,54181909,1320,5019,5022,7709,5,2018-07-05 04:55:14,2018-07-10 23:04:24,...,268.272783,2018-08-06 23:16:06,63.0,10.0,34.0,34.0,33.0,19.0,10,29.0
466,319993#4,4,56716871,1320,5019,5022,7709,3,2018-07-31 04:56:26,2018-08-06 23:16:06,...,243.661588,2018-08-31 00:00:00,26.0,6.0,1.0,1.0,19.0,-1.0,6,5.0


In [43]:
df_orders_orig

,item_sku_id,int_org_num,pur_bill_id,item_first_cate_cd,item_second_cate_cd,item_third_cate_cd,brand_code,create_day_of_week,create_tm,complete_dt,...,review_period,next_complete_dt,demand_RV,demand_V,target_decision,target_decision_nobc,overall_opt_order,inv,d_vlt,initial_stock_overall
0,258214#3,3,54484267,1320,5019,5022,4537,2,2018-07-09 05:16:27,2018-07-15 10:19:21,...,7.013715,2018-07-22 16:45:02,2204.0,1106.0,1214.0,1098.0,33.0,19.0,10,29.0
1,258214#3,3,55142334,1320,5019,5022,4537,2,2018-07-16 05:36:12,2018-07-22 16:45:02,...,6.984097,2018-07-29 14:11:37,2371.0,1098.0,739.0,739.0,19.0,-1.0,6,5.0
2,258214#3,3,55847850,1320,5019,5022,4537,2,2018-07-23 05:13:18,2018-07-29 14:11:37,...,6.984444,2018-08-04 16:34:00,2209.0,1273.0,1013.0,936.0,33.0,19.0,10,29.0
3,258214#3,3,56550600,1320,5019,5022,4537,2,2018-07-30 04:50:54,2018-08-04 16:34:00,...,1.417975,2018-08-04 16:27:58,936.0,936.0,21.0,0.0,19.0,-1.0,6,5.0
4,258214#3,3,56764306,1320,5019,5022,4537,3,2018-07-31 14:52:47,2018-08-04 16:27:58,...,7.062083,2018-08-14 15:46:14,2478.0,736.0,1685.0,1685.0,1341.0,0.0,555,555.0
5,258214#3,3,57501887,1320,5019,5022,4537,3,2018-08-07 16:22:11,2018-08-14 15:46:14,...,5.543704,2018-08-18 14:39:14,2038.0,1468.0,0.0,0.0,1048.0,-340.0,1226,886.0
6,258214#3,3,58077104,1320,5019,5022,4537,2,2018-08-13 05:25:07,2018-08-18 14:39:14,...,6.987188,2018-08-25 14:41:32,2244.0,1152.0,449.0,449.0,1015.0,0.0,1048,1048.0
7,258214#3,3,58870686,1320,5019,5022,4537,2,2018-08-20 05:06:40,2018-08-25 14:41:32,...,10.787037,2018-08-31 00:00:00,2051.0,955.0,355.0,355.0,1277.0,0.0,774,774.0
8,258214#4,4,54484260,1320,5019,5022,4537,2,2018-07-09 05:16:27,2018-07-18 23:17:14,...,7.013704,2018-07-25 23:25:03,450.0,249.0,411.0,201.0,379.0,-210.0,249,39.0
9,258214#4,4,55142241,1320,5019,5022,4537,2,2018-07-16 05:36:11,2018-07-25 23:25:03,...,13.968160,2018-08-08 23:09:21,559.0,253.0,559.0,306.0,286.0,0.0,221,221.0
